This Notebook runs HDC code on pynq board.
======
We compare the acuracy and time it takes to run HDC code on FPGA vs. HDC code with numpy.
Both version uses pre-populated basis and classes vectors. 

In [13]:
#Settings for the project
import pynq
from pynq import Overlay

import numpy as np
import time

program_start = time.time()
num_dims = 1024
num_features = 784
num_classes = 10
num_test = 100

#Grabbing the IP of wrapper function
overlay = Overlay("/home/xilinx/jupyter_notebooks/HDC/hdc_v6.bit")
encode_inst = overlay.hdc_wrapper_0


Load in the MNIST test data and pre-populated basis and classes vector
-----------
All data and vectors are stored in csv file.
The csv files are in HDC/DataBase folder

In [2]:
import csv
parsing_start = time.time()
all_test_data = []
all_label = []
#parse the test data
with open('/home/xilinx/jupyter_notebooks/HDC/DataBase/mnist_test.csv', 'r') as csvfile:
    allRow = csvfile.readlines()
    for i, data in enumerate(allRow):
        if i == 0: continue #skipping the header
        for j, elem in enumerate(data.split(',')):
            if j == 0: all_label.append(int(elem))
            else: all_test_data.append(int(elem))

In [3]:
#loading the pre-populated basis
basis_1d = [0]*(num_dims*num_features)
with open('/home/xilinx/jupyter_notebooks/HDC/DataBase/basis.csv', 'r') as file:
    csvfile = csv.reader(file)
    for line in csvfile:
        if len(line) > 0:
            basis_1d[:] = map(float, line)

In [4]:
#loading the pre-populated classes vector
classes_array = []
with open('/home/xilinx/jupyter_notebooks/HDC/DataBase/classes.csv', 'r') as file:
    csvfile = csv.reader(file)
    for line in csvfile:
        if len(line) > 0:
            classes_array.append(list(map(float, line)))
parsing_end = time.time()

Running the HDC code on FPGA
-----------
We first allocate memories for the input parameters. Then we populate the memories with correct inputs. After that we send the parameters over to the FPGA for calculation.

After the calculation is completed, we will run similarity test to see the accuracy of the results on classifying the images.

In [5]:
#Initialize
array_samples = pynq.allocate(shape=(num_test*num_features), dtype=np.float32)
array_basis   = pynq.allocate(shape=(num_dims*num_features), dtype=np.float32)
array_results = pynq.allocate(shape=(num_test*num_dims), dtype=np.float32)

In [6]:
array_samples[:] = all_test_data[:num_test*num_features]
array_basis[:]   = basis_1d
array_results[:] = np.zeros(shape=(num_test*num_dims,), dtype=np.float32)


array_samples.sync_to_device()
array_basis.sync_to_device()
array_results.sync_to_device()

In [7]:
encode_inst.mmio.write_reg(0x10, array_samples.physical_address)
encode_inst.mmio.write_reg(0x14, 0)
encode_inst.mmio.write_reg(0x1c, array_basis.physical_address)
encode_inst.mmio.write_reg(0x20, 0)
encode_inst.mmio.write_reg(0x28, array_results.physical_address)
encode_inst.mmio.write_reg(0x2c, 0)

In [8]:
#Run on FPGA
ip_status = encode_inst.read(0x00)
print(ip_status)

ip_status = encode_inst.read(0x00)
print(ip_status)

encode_inst.write(0x00, 1)
ip_status = encode_inst.read(0x00)
print(ip_status)
while (ip_status == 1):
    ip_status = encode_inst.read(0x00)

ip_status = encode_inst.read(0x00)
print(ip_status)
print('accelerator done')


4
4
1
4
accelerator done


In [9]:
ip_status = encode_inst.read(0x00)
print(ip_status)

#pulling result from function
array_samples.sync_from_device()
array_basis.sync_from_device()
array_results.sync_from_device()


4


In [10]:
import DataBase.similarity as sim
total_correct = 0
for i in range(num_test):
    start = i*num_dims
    end = start + num_dims
    encoded_test = array_results[start:end]
    result = sim.similarity(encoded_test, classes_array)
    if result == all_label[i]: total_correct += 1
print(f"Classified {num_test} images. Accuracy {total_correct}/{num_test} = {total_correct/num_test *100}%")
HDC_end = time.time()
print(f"Calculation using FPGA finished in {HDC_end - parsing_end} seconds")    

Classified 100 images. Accuracy 80/100 = 80.0%
Calculation using FPGA finished in 126.70011043548584 seconds


Running the HDC code with numpy
-----------

In [11]:
np_start = time.time()
#doing dot product with numpy
basis_2d = np.reshape(basis_1d, (num_features, num_dims), order='F')
np_total_correct = 0
for i in range(num_test):
    start = i*num_features
    end = start + num_features
    pased_sample = all_test_data[start:end]
    
    #doing dot product
    ref_encoded = np.dot(pased_sample, basis_2d)
    encoded_max = np.max(ref_encoded)
    encoded_min = np.min(ref_encoded)
    ref_encoded = -1.0 + (2.0 / (encoded_max - encoded_min)) * (ref_encoded - encoded_min)
    
    #similarity check
    result = sim.similarity(ref_encoded, classes_array)
    if result == all_label[i]: np_total_correct += 1
        
print(f"Classified {num_test} images. Accuracy {np_total_correct}/{num_test} = {np_total_correct/num_test *100}%")
np_end = time.time()
print(f"Calculation using numpy finished in {np_end - np_start} seconds")  

Classified 100 images. Accuracy 80/100 = 80.0%
Calculation using numpy finished in 14.992710590362549 seconds
